<a href="https://colab.research.google.com/github/onuralpArsln/finTech/blob/main/getDataFromBinanceWithoutAccount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gerekli Kütüphane yoksa yükle

In [1]:
!pip install python-binance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 12.7 MB/s eta 0:00:00


Datayı hesap bilgilisi olmadan almak burası, ancak colabda çalışmama ihtimali yüksek bi sebepten dolayı colab istekleri yetkisiz gözüküyor

In [3]:
import requests
import time


def getSevenDayData(lastDay=1):
    url = 'https://api.binance.com/api/v3/klines'
    current_time = int(time.time() * 1000)
    startDay = current_time - ((lastDay+7) * 24 * 60 * 60 * 1000)
    endDay = current_time - (lastDay * 24 * 60 * 60 * 1000)

    params = {
    'symbol': 'BTCUSDT',      # The trading pair you're interested in
    'interval': '1h',         # 1-hour candlestick data (hourly prices)
    'startTime': startDay,  # Data from 30 days ago
    'endTime': endDay,    # Data until 20 days ago
    }

    response = requests.get(url, params=params)
    data = response.json()

    result=[]

    for candle in data:
        close_price = candle[4]
        result.append(close_price)

    return result




test et

In [3]:
print(getSevenDayData())

IndexError: string index out of range

toplanan bilgiyi csv formatına çevirme kodu

In [7]:
import csv

data = getSevenDayData(28)


with open("data/fourWeek.csv", "w", newline="") as file:

    writer = csv.writer(file)
    writer.writerow(["data"]) 
    for item in data:
        writer.writerow([item])

csv bilgilerini numpy arraya çevirme kodu

In [ ]:
!pip install numpy -q

In [ ]:
import numpy as np

# Load CSV into numpy array
array_from_csv = np.loadtxt("data/lastWeek.csv", delimiter=",")

print(array_from_csv[0])
print(array_from_csv[2])
print(array_from_csv[3])


60138.14
60070.2
59851.09


Colab kullanırken csvleri otomatik çekme kodu

In [4]:
## Güncel data linkleri <br> <br>

file1 = "https://raw.githubusercontent.com/onuralpArsln/finTech/refs/heads/main/data/lastWeek.csv?token=GHSAT0AAAAAACLQZ7SITTNWYGU4AFMRQCZ2ZXPXZZA"
file2 = "https://raw.githubusercontent.com/onuralpArsln/finTech/refs/heads/main/data/twoWeek.csv?token=GHSAT0AAAAAACLQZ7SIAZ6M2GER76YGBQ4EZXPXZ3A"
file3 = "https://raw.githubusercontent.com/onuralpArsln/finTech/refs/heads/main/data/fourWeek.csv?token=GHSAT0AAAAAACLQZ7SIZUNDS5LREKVI4BA4ZXPXY2Q"

allLinks =[file1, file2, file3]

In [5]:
!pip install pandas -q

In [6]:
import pandas as pd

csv_url = file1
df1 = pd.read_csv(csv_url)
print(df1.head())

dfs=[]
for link in allLinks:
    dfs.append(pd.read_csv(link))


   60138.14000000
0        59973.99
1        60070.20
2        59851.09
3        59706.39
4        59794.03


linklerden elde edilen dataframeleri eğitim ve test olarak ayırcaz son 10 test datası şuan

In [7]:
test_dfs=[]
train_dfs=[]

for df in dfs:
    # Separate last 10 rows
    last_10 = df.tail(10)
    # Separate the rest of the rows
    rest = df.iloc[:-10]
    # append to dataset
    test_dfs.append(last_10)
    train_dfs.append(rest)



## TRAINING TRAINING TRAINING


Autoregressive Integrated Moving Average

In [8]:
!pip install statsmodels -q

In [13]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

"""
# Örnek zaman serisi verisi
data = [1,2,4,7]
df = pd.Series(data)

# ARIMA modeli
model = ARIMA(df, order=(1,1,1))
model_fit = model.fit()

# Tahmin
tahmin = model_fit.forecast(steps=1)
print(tahmin)
"""

def ARIMAGuess(dataSeries,stepCount=10):
    df = pd.Series(data)
    model = ARIMA(df, order=(1,1,1))
    model_fit = model.fit()
    tahmin = model_fit.forecast(steps=stepCount)
    return tahmin


In [14]:
guess_dsf=[]

for df in train_dfs:
    series = df['data']
    ARIMA_guess = ARIMAGuess(series,10)
    guess_dsf.append(ARIMA_guess)


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().